In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix, classification_report
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
dfs, available_classes, available_models = read_raw_reports(os.path.join('..',raw_reports_path))

# replacing spaces with underscores:
available_classes = [cname.replace(' ', '_') for cname in available_classes]

In [3]:
# having it reverse for plot:
available_classes_r = available_classes[::-1]

available_classes_r_upper = [cname.upper() for cname in available_classes_r]

In [4]:
available_models

['ViT-bigG-14-CLIPA-336',
 'ViT-L_14',
 'ViT-SO400M-14-SigLIP-384',
 'ViT-H-14-378-quickgelu',
 'RN50x64',
 'ViT-B_32',
 'EVA02-E-14-plus',
 'RN101']

In [5]:
available_classes

['asphalt',
 'concrete',
 'concrete_plates',
 'grass',
 'ground',
 'paving_stones',
 'sett']

In [6]:
all_dfs = []
modelwise_dfs = {}

for class_dict in dfs:
    for model_name in dfs[class_dict]:
        model_df = dfs[class_dict][model_name]
        all_dfs.append(model_df)

        if not model_name in modelwise_dfs:
            modelwise_dfs[model_name] = []

        modelwise_dfs[model_name].append(model_df)


for model_name in modelwise_dfs:
    modelwise_dfs[model_name] = pd.concat(modelwise_dfs[model_name]).copy()

In [7]:
{model_name: len(modelwise_dfs[model_name]) for model_name in modelwise_dfs}

{'EVA02-E-14-plus': 2619,
 'RN101': 2619,
 'RN50x64': 2619,
 'ViT-bigG-14-CLIPA-336': 2619,
 'ViT-B_32': 2619,
 'ViT-H-14-378-quickgelu': 2619,
 'ViT-L_14': 2619,
 'ViT-SO400M-14-SigLIP-384': 2619}

In [8]:
merged_df = pd.concat(all_dfs)

In [9]:
true_classes = merged_df.groupby('image_name').agg('first')[class_label] #.str.replace('_', ' ')
merged_pred = merged_df.groupby('image_name').sum(numeric_only=True)

merged_pred[prediction_label] = merged_pred.idxmax(axis=1)
merged_pred[class_label] = true_classes


In [10]:
merged_pred

,asphalt,concrete,concrete_plates,grass,ground,paving_stones,sett,prediction,class
image_name,,,,,,,,,
00098db7c6cd6ee36d590ff0d5db7ad1.png,0.213754,3.985507,0.961183,0.019217,0.534829,1.280490,0.005109,concrete,concrete
00157343508e08f1dbc0efc1f8066f3d.png,0.601964,0.216773,0.454248,0.994805,2.932286,1.649854,0.149998,ground,ground
003a08eff81fe589f95102bf2ce1ecf5.png,0.007631,0.002122,0.013925,5.955048,0.965288,0.043469,0.012785,grass,grass
005b94f55127ae460c0741eac3a89b25.png,4.938634,0.277412,1.465858,0.012804,0.181552,0.108702,0.015524,asphalt,asphalt
006d04cdfa79fac8f108a0a1cd34d0f8.png,0.018306,0.004128,0.016005,6.087209,0.791766,0.057336,0.025230,grass,grass
...,...,...,...,...,...,...,...,...,...
ffbd835981460b0e1b7825681c7b9449.png,0.544958,0.495025,0.475408,0.098204,0.890578,3.950697,0.545128,paving_stones,concrete_plates
ffd0ab8395fc05d8d14f3cfe8a6a6b4a.png,0.600495,0.556082,0.496194,0.633679,3.802484,0.742776,0.168145,ground,ground
ffd4bc27fe750ca5fedac61da4f7a397.png,0.428585,0.148436,0.752849,0.359114,3.330605,1.677660,0.302741,ground,ground


In [11]:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,
                         predicted_labels,
                         labels=available_classes_r,
                           normalize='true'
                           ) * 100
c_mat = c_mat.round(3)

In [12]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [13]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes_r,
    output_dict=True
)

{'sett': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 346.0},
 'paving_stones': {'precision': 0.4550561797752809,
  'recall': 0.9788519637462235,
  'f1-score': 0.6212847555129435,
  'support': 331.0},
 'ground': {'precision': 0.7236024844720497,
  'recall': 0.9872881355932204,
  'f1-score': 0.8351254480286738,
  'support': 472.0},
 'grass': {'precision': 0.9166666666666666,
  'recall': 0.9890350877192983,
  'f1-score': 0.9514767932489452,
  'support': 456.0},
 'concrete_plates': {'precision': 0.46551724137931033,
  'recall': 0.15428571428571428,
  'f1-score': 0.2317596566523605,
  'support': 175.0},
 'concrete': {'precision': 0.6896551724137931,
  'recall': 0.7253886010362695,
  'f1-score': 0.7070707070707071,
  'support': 386.0},
 'asphalt': {'precision': 0.839041095890411,
  'recall': 0.5408388520971302,
  'f1-score': 0.6577181208053692,
  'support': 453.0},
 'accuracy': 0.6846124474990455,
 'macro avg': {'precision': 0.584219834371073,
  'recall': 0.6250983363539795

In [14]:
n_rows = int(ceil(len(available_models)/2))

In [15]:
modelwise_confusion_matrices = {}

for model in modelwise_dfs:
    true_labels =      modelwise_dfs[model][class_label]#.tolist()
    predicted_labels = modelwise_dfs[model][prediction_label]#.tolist()

    modelwise_confusion_matrices[model] = confusion_matrix(
        true_labels,
        predicted_labels,
        labels=available_classes,
        normalize='true'
        ) *  100
    modelwise_confusion_matrices[model] = modelwise_confusion_matrices[model].round(3)

In [16]:
modelwise_confusion_matrices

{'EVA02-E-14-plus': array([[59.603, 28.256,  0.883,  5.077,  3.753,  0.   ,  2.428],
        [17.098, 72.539,  0.777,  3.109,  5.44 ,  0.259,  0.777],
        [ 9.714, 32.   ,  4.571,  8.   , 24.571, 21.143,  0.   ],
        [ 0.439,  0.219,  0.   , 98.465,  0.877,  0.   ,  0.   ],
        [15.042,  1.907,  0.   ,  1.483, 77.754,  0.847,  2.966],
        [ 4.834,  0.906,  0.   ,  0.   ,  2.115, 92.145,  0.   ],
        [ 4.046,  3.179,  0.   ,  1.734, 11.85 , 79.191,  0.   ]]),
 'RN101': array([[23.841,  0.221, 37.969,  4.636, 19.426,  5.077,  8.83 ],
        [ 2.591,  0.777, 64.767,  0.259,  5.699, 20.207,  5.699],
        [ 0.571,  0.571, 50.286,  0.571, 10.286, 37.714,  0.   ],
        [ 0.   ,  0.   ,  0.877, 84.211, 14.254,  0.219,  0.439],
        [ 0.424,  0.   ,  5.508,  1.483, 76.271, 15.89 ,  0.424],
        [ 0.604,  0.   ,  3.625,  0.   ,  0.604, 94.864,  0.302],
        [ 2.312,  0.   ,  5.78 ,  0.578,  4.046, 87.283,  0.   ]]),
 'RN50x64': array([[ 9.934, 45.475,  0.221, 

In [17]:
# fig = go.Figure().set_subplots(rows=n_rows, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.05, horizontal_spacing=0.01)
fig = make_subplots(rows=n_rows, cols=2, 
                    shared_xaxes=True, 
                    shared_yaxes=True,
                    subplot_titles=list(modelwise_confusion_matrices.keys()),
                    vertical_spacing=0.015, 
                    horizontal_spacing=0.01,
                    # x_title="Prediction",
                    # y_title="Class",
                    )

for i,modelname in enumerate(modelwise_confusion_matrices):
    subplot = px.imshow(modelwise_confusion_matrices[modelname],
                
                x=available_classes,
                y=available_classes,
              text_auto=True, 
              #  aspect="auto",
               )
    
    # print(i//2+1,i%2+1)
    subplot['layout']['xaxis']['autorange'] = "reversed"

    fig.add_trace(subplot.data[0], row=i//2+1, col=i%2+1)
    fig.update_xaxes(row=i//2+1, col=i%2+1, autorange='reversed')
    # fig.append_trace(subplot.data[0], row=i//2+1, col=i%2+1)

fig.update_coloraxes(colorscale=px.colors.sequential.Burg)
fig.update_layout(height=400*n_rows, width=400*2,
                  xaxis_title="Prediction",
                  yaxis_title="Class",
                  showlegend=True,)
fig.show()


In [18]:
# getting the diagonal sum of the confusion matrix:
diagonals = {}

for model in modelwise_confusion_matrices:
    diagonals[model] = modelwise_confusion_matrices[model].diagonal().sum()

diagonals['combined'] = c_mat.diagonal().sum() 

diagonals_df = pd.DataFrame.from_dict(diagonals, orient='index',columns=['score']).reset_index().rename(columns={'index':'model_name'}).sort_values(by='score', ascending=False)

diagonals_df['score'] = round(100 *  diagonals_df['score'] / (len(available_models) * 100),2)

diagonals_df

,model_name,score
8,combined,54.70
5,ViT-H-14-378-quickgelu,53.01
3,ViT-bigG-14-CLIPA-336,50.82
0,EVA02-E-14-plus,50.63
7,ViT-SO400M-14-SigLIP-384,49.64
6,ViT-L_14,42.67
1,RN101,41.28
4,ViT-B_32,36.82
2,RN50x64,33.15


In [19]:
fig = px.bar(diagonals_df,x='model_name', y='score', text_auto=True)
fig.show()


In [20]:
selected_models = {}

for modelname in ['ViT-H-14-378-quickgelu', 'ViT-SO400M-14-SigLIP-384', 'EVA02-E-14-plus']:
    selected_models[modelname] = modelwise_dfs[modelname]

In [21]:
merged_selected = pd.concat(selected_models).reset_index()

In [22]:
true_classes = merged_selected.groupby('image_name').agg('first')[class_label] #.str.replace('_', ' ') #.reset_index().rename(columns={class_label:'class'})

In [23]:
merged_selected = merged_selected.groupby('image_name').sum(numeric_only=True)

In [24]:
merged_selected.drop(columns=['level_1'], inplace=True)

In [25]:
merged_pred[prediction_label] = merged_selected.idxmax(axis=1)
merged_pred[class_label] = true_classes.tolist()

In [26]:
# computing the confusion matrix:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r,
                          normalize='true'
                          ) * 100
c_mat = c_mat.round(3)

In [27]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [28]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes_r,
    output_dict=True
)

{'sett': {'precision': 0.46153846153846156,
  'recall': 0.017341040462427744,
  'f1-score': 0.033426183844011144,
  'support': 346.0},
 'paving_stones': {'precision': 0.4960380348652932,
  'recall': 0.945619335347432,
  'f1-score': 0.6507276507276507,
  'support': 331.0},
 'ground': {'precision': 0.7444444444444445,
  'recall': 0.9936440677966102,
  'f1-score': 0.8511796733212341,
  'support': 472.0},
 'grass': {'precision': 0.9356846473029046,
  'recall': 0.9890350877192983,
  'f1-score': 0.9616204690831557,
  'support': 456.0},
 'concrete_plates': {'precision': 0.7358490566037735,
  'recall': 0.22285714285714286,
  'f1-score': 0.34210526315789475,
  'support': 175.0},
 'concrete': {'precision': 0.6531049250535332,
  'recall': 0.7901554404145078,
  'f1-score': 0.7151230949589683,
  'support': 386.0},
 'asphalt': {'precision': 0.7784256559766763,
  'recall': 0.5894039735099338,
  'f1-score': 0.6708542713567839,
  'support': 453.0},
 'accuracy': 0.7063764795723558,
 'macro avg': {'preci

In [35]:
c_mat.diagonal().sum() / (len(available_models))

56.85075